# Captions data
Notebook to create a captions dataframe that will be used by ModernBERT to embed text.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from fungiclef.config import get_device

device = get_device()
device

'cuda'

In [3]:
import os
from pathlib import Path

# get list of stored filed in cloud bucket
root = Path(os.path.expanduser("~"))
! date

Sun May 11 20:14:56 EDT 2025


In [4]:
import pandas as pd


data_path = f"{root}/p-dsgt_clef2025-0/shared/fungiclef/data"
train_path = f"{data_path}/dataset/processed/train_serialized.parquet"
val_path = f"{data_path}/dataset/processed/val_serialized.parquet"
test_path = f"{data_path}/dataset/processed/test_serialized.parquet"
train_df = pd.read_parquet(train_path)
val_df = pd.read_parquet(val_path)
test_df = pd.read_parquet(test_path)
train_df.head()

,eventDate,year,month,day,habitat,countryCode,scientificName,kingdom,phylum,class,...,region,district,filename,category_id,metaSubstrate,poisonous,elevation,landcover,biogeographicalRegion,data
0,2021-01-24,2021,1.0,24.0,Mixed woodland (with coniferous and deciduous ...,DK,Xylohypha ferruginosa (Corda) S.Hughes,Fungi,Ascomycota,Eurotiomycetes,...,Sjælland,Næstved,0-3052832307.JPG,2421,wood,0,0.0,16.0,continental,b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00...
1,2021-02-27,2021,2.0,27.0,garden,DK,"Comatricha alta Preuss, 1851",Protozoa,Mycetozoa,Myxomycetes,...,Hovedstaden,Gribskov,0-3061954303.JPG,386,wood,0,0.0,17.0,continental,b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00...
2,2021-02-27,2021,2.0,27.0,garden,DK,"Comatricha alta Preuss, 1851",Protozoa,Mycetozoa,Myxomycetes,...,Hovedstaden,Gribskov,1-3061954303.JPG,386,wood,0,0.0,17.0,continental,b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00...
3,2021-02-27,2021,2.0,27.0,garden,DK,"Comatricha alta Preuss, 1851",Protozoa,Mycetozoa,Myxomycetes,...,Hovedstaden,Gribskov,2-3061954303.JPG,386,wood,0,0.0,17.0,continental,b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00...
4,2021-02-27,2021,2.0,27.0,garden,DK,"Comatricha alta Preuss, 1851",Protozoa,Mycetozoa,Myxomycetes,...,Hovedstaden,Gribskov,3-3061954303.JPG,386,wood,0,0.0,17.0,continental,b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00...


In [5]:
train_df.columns, len(train_df.columns.to_list())

(Index(['eventDate', 'year', 'month', 'day', 'habitat', 'countryCode',
        'scientificName', 'kingdom', 'phylum', 'class', 'order', 'family',
        'genus', 'specificEpithet', 'hasCoordinate', 'species',
        'iucnRedListCategory', 'substrate', 'latitude', 'longitude',
        'coorUncert', 'observationID', 'region', 'district', 'filename',
        'category_id', 'metaSubstrate', 'poisonous', 'elevation', 'landcover',
        'biogeographicalRegion', 'data'],
       dtype='object'),
 32)

In [6]:
val_df.columns, len(val_df.columns.to_list())

(Index(['eventDate', 'year', 'month', 'day', 'habitat', 'countryCode',
        'scientificName', 'kingdom', 'phylum', 'class', 'order', 'family',
        'genus', 'specificEpithet', 'hasCoordinate', 'species',
        'iucnRedListCategory', 'substrate', 'latitude', 'longitude',
        'coorUncert', 'observationID', 'region', 'district', 'filename',
        'category_id', 'metaSubstrate', 'poisonous', 'elevation', 'landcover',
        'biogeographicalRegion', 'data'],
       dtype='object'),
 32)

In [7]:
test_df.columns, len(test_df.columns.to_list())

(Index(['eventDate', 'year', 'month', 'day', 'habitat', 'countryCode',
        'hasCoordinate', 'substrate', 'latitude', 'longitude', 'coorUncert',
        'observationID', 'region', 'district', 'filename', 'metaSubstrate',
        'elevation', 'landcover', 'biogeographicalRegion', 'data'],
       dtype='object'),
 20)

In [8]:
test_df[
    [
        "substrate",
        "latitude",
        "longitude",
        "coorUncert",
        "observationID",
        "region",
        "district",
        "filename",
        "metaSubstrate",
        "elevation",
        "landcover",
        "biogeographicalRegion",
    ]
].iloc[0]

substrate                dead wood (including bark)
latitude                                  55.993269
longitude                                 12.290225
coorUncert                                     50.0
observationID                            4100099350
region                                  Hovedstaden
district                                   Hillerød
filename                           0-4100099350.JPG
metaSubstrate                                  wood
elevation                                      43.0
landcover                                      12.0
biogeographicalRegion                   continental
Name: 0, dtype: object

In [9]:
# read json captions into dataframe
import json
import pandas as pd
import os


def get_json_files(json_path):
    return [
        os.path.join(json_path, file)
        for file in os.listdir(json_path)
        if file.endswith(".json")
    ]


def read_json_file(file_path):
    with open(file_path, "r") as f:
        data = json.load(f)
    return data


def json_to_dataframe(json_data):
    df = pd.DataFrame(json_data)
    return df


# main function to get json files into dataframe
def get_json_df(json_path):
    json_files = get_json_files(json_path)
    records = []
    for file_path in json_files:
        filename = os.path.basename(file_path).replace(".json", "")
        data = read_json_file(file_path)
        caption = data  # data is a raw string
        records.append({"filename": filename, "caption": caption})
    return pd.DataFrame(records)


train_json_path = f"{data_path}/dataset/captions/train"
val_json_path = f"{data_path}/dataset/captions/val"
test_json_path = f"{data_path}/dataset/captions/test"

train_json_df = get_json_df(train_json_path)
val_json_df = get_json_df(val_json_path)
test_json_df = get_json_df(test_json_path)

In [10]:
train_json_df.head()

,filename,caption
0,0-2237953479.JPG,The image shows a close-up view of a tree tru...
1,3-3414824336.JPG,The image shows a piece of wood with a promin...
2,5-2238512677.JPG,The image shows a fascinating close-up of a f...
3,1-2999542304.JPG,"The image shows several small, dark-colored f..."
4,1-2238522794.JPG,The image shows two mushrooms on a forest flo...


In [11]:
# write json captions to parquet
def write_json_to_parquet(json_df, output_path):
    json_df.to_parquet(output_path)
    print(f"Captions dataframe saved to {output_path}")


train_json_output_path = f"{data_path}/dataset/processed/train_captions.parquet"
val_json_output_path = f"{data_path}/dataset/processed/val_captions.parquet"
test_json_output_path = f"{data_path}/dataset/processed/test_captions.parquet"

write_json_to_parquet(train_json_df, train_json_output_path)
write_json_to_parquet(val_json_df, val_json_output_path)
write_json_to_parquet(test_json_df, test_json_output_path)

Captions dataframe saved to /storage/home/hcoda1/9/mgustineli3/p-dsgt_clef2025-0/shared/fungiclef/data/dataset/processed/train_captions.parquet
Captions dataframe saved to /storage/home/hcoda1/9/mgustineli3/p-dsgt_clef2025-0/shared/fungiclef/data/dataset/processed/val_captions.parquet
Captions dataframe saved to /storage/home/hcoda1/9/mgustineli3/p-dsgt_clef2025-0/shared/fungiclef/data/dataset/processed/test_captions.parquet
